In [14]:
import pandas as pd
import os
from dates import extract_dates_from_text


Fetch data from hugging face (only run once ✅)

In [ ]:
# df = pd.read_parquet("hf://datasets/RevolutionCrossroads/nara_revolutionary_war_pension_files/nara_pension_file_pages.parquet")


In [5]:
# Save the dataframe locally to avoid re-downloading
# df.to_parquet('nara_pension_file_pages.parquet', engine='pyarrow')
# print(f"Dataframe saved locally with {len(df)} rows and {len(df.columns)} columns")
# print(f"File size: {os.path.getsize('nara_pension_file_pages.parquet') / (1024*1024):.2f} MB")


Dataframe saved locally with 2244629 rows and 19 columns
File size: 1211.39 MB


Use locally saved version of original dataaset

In [ ]:
# df = pd.read_parquet('nara_pension_file_pages.parquet')


Add a new column to save dates found in ocrText column

In [8]:
df['ocrTextDates'] = None

In [9]:
df.head()

,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,ocrID,ocrText,ocrUploadDate,ocrContributor,transcriptionID,transcriptionText,transcriptionContributionCount,transcriptionUserNames,transcriptionDate,ocrTextDates
0,111769430,https://catalog.archives.gov/id/111769430,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769431,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850379,Mass\nWoodbury\nWm\nS33948,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,None
1,111769430,https://catalog.archives.gov/id/111769430,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769432,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850380,33948\nRevy INVALID.\nFile No. 33948\nIndex\nW...,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,None
2,111769430,https://catalog.archives.gov/id/111769430,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769433,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850381,To the Secretary of the Department of War\nSir...,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,None
3,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769419,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850368,"(3 - 1776)\nSERVICE\nMass\nCONTENTS\nWoodbury,...",2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,None
4,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2018"", ""type"": ""Search Iden...",None,None,111769420,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,Image (JPG),19850369,1831\nto\nSchaled Tucker\n1941\nMassachusetts\...,2024-11-20T20:32:03.000Z,FamilySearch,None,None,NaN,None,None,None


In [10]:
# Search for "eighteen" in the ocrText column
search_term = "eighteen"
mask = df['ocrText'].str.contains(search_term, case=False, na=False)
results = df[mask]

print(f"Found {len(results)} rows containing '{search_term}'")
print(f"Percentage of total rows: {len(results)/len(df)*100:.2f}%")

# Show first few results
if len(results) > 0:
    print(f"\nFirst 5 results:")
    print(results[['ocrText']].head())
else:
    print("No results found")


Found 80068 rows containing 'eighteen'
Percentage of total rows: 3.57%

First 5 results:
                                               ocrText
8    and at the same time he produces to the Court ...
30   Rock\nState of New Hampshire\nSingham ss\nJ. S...
92   Grafton\nassociate\nNew Ha\nand that\nhis is\n...
93   State of New Hampshire\nCounty of Grafton ⎬ SS...
115  AFFIDAVIT OF WITNESSES.\n---\nSTATE OF MASSACH...


In [15]:
# Test the date extraction function
test_texts = [
    "The year was 1840 when he was born",
    "eighteen forty was a good year",
    "eighteen and forty",
    "nineteen hundred and twenty",
    "He died in 1750 and was born in eighteen twenty",
    "No dates here",
    "The war ended in eighteen sixty five"
]

print("Testing date extraction function:")
for text in test_texts:
    dates = extract_dates_from_text(text)
    print(f"'{text}' -> {dates}")


Testing date extraction function:
'The year was 1840 when he was born' -> ['1840']
'eighteen forty was a good year' -> []
'eighteen and forty' -> []
'nineteen hundred and twenty' -> []
'He died in 1750 and was born in eighteen twenty' -> ['1750']
'No dates here' -> []
'The war ended in eighteen sixty five' -> []


In [ ]:
# Apply date extraction to the entire dataframe
print("Extracting dates from ocrText column...")
print(f"Processing {len(df)} rows...")

# Apply the function to each row
df['ocrTextDates'] = df['ocrText'].apply(extract_dates_from_text)

# Show statistics
total_dates = sum(len(dates) for dates in df['ocrTextDates'])
rows_with_dates = len(df[df['ocrTextDates'].apply(len) > 0])

print(f"\nDate extraction complete!")
print(f"Total dates found: {total_dates}")
print(f"Rows with dates: {rows_with_dates}")
print(f"Rows without dates: {len(df) - rows_with_dates}")
print(f"Average dates per row: {total_dates/len(df):.2f}")

# Show some examples
print(f"\nFirst 10 rows with dates:")
examples = df[df['ocrTextDates'].apply(len) > 0].head(10)
for idx, row in examples.iterrows():
    print(f"Row {idx}: {row['ocrTextDates']} - Text: {str(row['ocrText'])[:100]}...")


In [17]:
# Create a NAID-collapsed version of the dataframe and save locally (per custom rules)
from collections.abc import Sequence

# Ensure NAID is numeric (confirming it's numbers only); keep missing as <NA>
df['NAID'] = pd.to_numeric(df['NAID'], errors='coerce').astype('Int64')

# Helper: treat empty strings (after strip) as missing
def _is_missing_value(v):
    if pd.isna(v):
        return True
    if isinstance(v, str) and v.strip() == '':
        return True
    return False

# Collapse/collect per group, ordering by pageObjectId
# Rules:
# - If after excluding missing/empty, all values equal -> return that scalar
# - Else collect ALL non-missing values in row-order sorted by pageObjectId
# - For list/tuple items, concatenate (no de-duplication)
# - Keep duplicates in collected arrays
def collapse_or_collect_for_group(group_df: pd.DataFrame, column_name: str):
    # Sort rows by pageObjectId to define deterministic order
    if 'pageObjectId' in group_df.columns:
        ordered = group_df.sort_values('pageObjectId')
    else:
        ordered = group_df

    values = ordered[column_name].tolist()
    non_missing_values = [v for v in values if not _is_missing_value(v)]

    if len(non_missing_values) == 0:
        return pd.NA

    # Determine uniqueness of non-missing scalars/lists via string representation for equality
    # If all non-missing values serialize identically, collapse to the first (scalar or list)
    first_val = non_missing_values[0]
    if all((repr(v) == repr(first_val)) for v in non_missing_values):
        return first_val

    # Otherwise collect in page order, flatten lists/tuples, keep duplicates
    collected = []
    for v in non_missing_values:
        if isinstance(v, (list, tuple)):
            collected.extend(list(v))
        else:
            collected.append(v)
    return collected

print("Collapsing rows by NAID with custom rules ...")

# Columns to include: same as original df, but we will drop ocrTextDates in the result per instructions
all_columns = list(df.columns)

# Group and aggregate
grouped = df.groupby('NAID', dropna=False, as_index=False)
rows = []
for naid_val, g in grouped:
    row_dict = {}
    for col in all_columns:
        if col == 'ocrTextDates':
            # Drop this column in the new df
            continue
        row_dict[col] = collapse_or_collect_for_group(g, col)
    rows.append(row_dict)

collapsed_df = pd.DataFrame(rows)

print(f"Original rows: {len(df)} | Collapsed rows: {len(collapsed_df)}")


Collapsing rows by NAID with custom rules ...
Original rows: 2244629 | Collapsed rows: 78926


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [18]:
collapsed_df 

,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,ocrID,ocrText,ocrUploadDate,ocrContributor,transcriptionID,transcriptionText,transcriptionContributionCount,transcriptionUserNames,transcriptionDate
0,300036,https://catalog.archives.gov/id/300036,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""number"": ""NWCTB-15-NM22E19-R11789-SLIDE115""...",28238881,https://s3.amazonaws.com/NARAprodstorage/lz/re...,14989172,https://s3.amazonaws.com/NARAprodstorage/lz/re...,Image (JPG),20747708,J\nNames\nJoseph Wood and\nJoanna Tuttle\nD. J...,2024-11-22T02:30:45.000Z,FamilySearch,53ca038c-0628-42ce-b021-e9c46361c508,FAMILY\nJ. RECORD W.\nNames ᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧᐧ...,7.0,mpage77;willowtreeblossoms,2023-08-06 16:03:33
1,300040,https://catalog.archives.gov/id/300040,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""number"": ""NWCTB-15-NM22E19-R5568-SLIDE27"", ...",24519842,https://s3.amazonaws.com/NARAprodstorage/lz/re...,"[24519840, 24519841]",[https://s3.amazonaws.com/NARAprodstorage/lz/r...,Image (JPG),"[20747709, 20747710]","[ORD\nAlice\nAll\n⎬\n2\n-\nS\nJohn Jenkins, Bo...",2024-11-22T02:30:45.000Z,FamilySearch,"[3be1b37a-62ee-4a5d-8f9c-c3b020f6cd03, ca5f262...","[FAMILY RECORD\n\nJohn Jenkins, born Nov. 30t...","[6.0, 5.0]",[needhamgarden2285424;jpo.vox731422;sfrenck666...,"[2025-06-16 18:11:46, 2025-01-23 00:22:13]"
2,300070,https://catalog.archives.gov/id/300070,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""note"": ""This is the NARA internal exhibit t...",14990014,https://s3.amazonaws.com/NARAprodstorage/opast...,"[14990013, 14990015]",[https://s3.amazonaws.com/NARAprodstorage/opas...,Image (JPG),"[20747711, 20747712]","['-\n℗\nB\nDavid Rogers and\nRhoda Osborn, his...",2024-11-22T02:30:45.000Z,FamilySearch,"[f8b45815-dff8-346b-8466-06a67b3143f0, 411f748...",[[whole interspersed with hand drawn pen decor...,1.0,Catherine1,"[2016-07-15 20:58:25, 2016-07-15 21:01:08]"
3,300072,https://catalog.archives.gov/id/300072,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""number"": ""NWCTB-15-NM22E19-W15214-SLIDE59"",...",48118979,https://s3.amazonaws.com/NARAprodstorage/opast...,48118980,https://s3.amazonaws.com/NARAprodstorage/opast...,Image (JPG),20747713,'-\n-\n-\n---\n---\nto\n-\n-\nto\n℗\nAuby\nNAM...,2024-11-22T02:30:45.000Z,FamilySearch,88200bb2-dc85-3012-9476-7384aed13474,FAMILY RECORD\nO'Cranby's C.P.S.\nIn...,9.0,mpage77,2020-10-25 08:53:51
4,300092,https://catalog.archives.gov/id/300092,Fraktur of the Family of Peter Hunt,<NA>,"[{""note"": ""This is the NARA Internal Exhibit T...",<NA>,<NA>,16411580,https://s3.amazonaws.com/NARAprodstorage/opast...,Image (JPG),20747714,I\n8\nto\nSarah Cary\nPeter Hunt-\nHannah Hunt...,2024-11-22T02:30:45.000Z,FamilySearch,f2fdfba6-1c0f-37dd-9f12-37cc6e28dcff,Births. marriages. ...,2.0,ScribeOfPoland,2016-08-10 05:06:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78921,245050683,https://catalog.archives.gov/id/245050683,Copy of the Affidavit of Samuel Cherry of Pari...,1807-02-26,"[{""number"": ""HD1-730357612"", ""type"": ""HMS Asse...",267322352,https://s3.amazonaws.com/NARAprodstorage/lz/dc...,"[267322353, 267322354, 267322355]",[https://s3.amazonaws.com/NARAprodstorage/lz/d...,Image (JPG),"[20747693, 20747694, 20747695]",[I Samuel Cherry now of the County of Oneida T...,2024-11-22T02:30:45.000Z,FamilySearch,"[8b78a52d-7625-4bde-81c4-8dbb7a2a3fee, be955d3...",[I Samuel Cherry now of the County of Oneida T...,"[3.0, 3.0, 2.0]","[mlundblad;engadean390608;NPSkjwojack, mlundbl...","[2025-01-23 19:12:54, 2024-03-26 03:07:15, 202..."
78922,245050686,https://catalog.archives.gov/id/245050686,Copy of the Commission Issued to Adam Comstock...,1806-12-08,"[{""number"": ""HD1-730357621"", ""type"": ""HMS Asse...",267322356,https://s3.amazonaws.com/NARAprodstorage/lz/dc...,"[267322357, 267322358]",[https://s3.amazonaws.com/NARAprodstorage/lz/d...,Image (JPG),"[20747696, 207

In [25]:
import json
import numpy as np

def _is_listlike(v):
    return isinstance(v, (list, tuple))

def _is_missing_scalar(v):
    if _is_listlike(v):
        return False
    if isinstance(v, str):
        return v.strip() == ''
    try:
        return pd.isna(v)
    except Exception:
        return False

def _coerce_json(x):
    if _is_missing_scalar(x):
        return None
    if isinstance(x, (np.integer,)):
        return int(x)
    if isinstance(x, (np.floating,)):
        # keep floats as floats (even if like 1.0)
        return float(x)
    if isinstance(x, (np.bool_,)):
        return bool(x)
    return x

# Columns that contain any lists/tuples
list_cols = [c for c in collapsed_df.columns if collapsed_df[c].apply(_is_listlike).any()]
print("List-like columns to serialize:", list_cols)

df_to_save = collapsed_df.copy()

# Serialize ALL non-missing values in these columns as JSON array strings
for c in list_cols:
    df_to_save[c] = df_to_save[c].apply(
        lambda v: (
            pd.NA if _is_missing_scalar(v) else
            json.dumps([_coerce_json(x) for x in v]) if _is_listlike(v)
            else json.dumps([_coerce_json(v)])
        )
    )

List-like columns to serialize: ['pageObjectId', 'pageURL', 'ocrID', 'ocrText', 'ocrUploadDate', 'transcriptionID', 'transcriptionText', 'transcriptionContributionCount', 'transcriptionUserNames', 'transcriptionDate']


In [26]:
df_to_save

,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,ocrID,ocrText,ocrUploadDate,ocrContributor,transcriptionID,transcriptionText,transcriptionContributionCount,transcriptionUserNames,transcriptionDate
0,300036,https://catalog.archives.gov/id/300036,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""number"": ""NWCTB-15-NM22E19-R11789-SLIDE115""...",28238881,https://s3.amazonaws.com/NARAprodstorage/lz/re...,"[""14989172""]","[""https://s3.amazonaws.com/NARAprodstorage/lz/...",Image (JPG),"[""20747708""]","[""J\nNames\nJoseph Wood and\nJoanna Tuttle\nD....","[""2024-11-22T02:30:45.000Z""]",FamilySearch,"[""53ca038c-0628-42ce-b021-e9c46361c508""]","[""FAMILY\nJ. RECORD W.\nNames \u1427\u1427\u...",[7.0],"[""mpage77;willowtreeblossoms""]","[""2023-08-06 16:03:33""]"
1,300040,https://catalog.archives.gov/id/300040,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""number"": ""NWCTB-15-NM22E19-R5568-SLIDE27"", ...",24519842,https://s3.amazonaws.com/NARAprodstorage/lz/re...,"[""24519840"", ""24519841""]","[""https://s3.amazonaws.com/NARAprodstorage/lz/...",Image (JPG),"[""20747709"", ""20747710""]","[""ORD\nAlice\nAll\n\u23ac\n2\n-\nS\nJohn Jenki...","[""2024-11-22T02:30:45.000Z""]",FamilySearch,"[""3be1b37a-62ee-4a5d-8f9c-c3b020f6cd03"", ""ca5f...","[""FAMILY RECORD\n\nJohn Jenkins, born Nov. 30...","[6.0, 5.0]","[""needhamgarden2285424;jpo.vox731422;sfrenck66...","[""2025-06-16 18:11:46"", ""2025-01-23 00:22:13""]"
2,300070,https://catalog.archives.gov/id/300070,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""note"": ""This is the NARA internal exhibit t...",14990014,https://s3.amazonaws.com/NARAprodstorage/opast...,"[""14990013"", ""14990015""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),"[""20747711"", ""20747712""]","[""'-\n\u2117\nB\nDavid Rogers and\nRhoda Osbor...","[""2024-11-22T02:30:45.000Z""]",FamilySearch,"[""f8b45815-dff8-346b-8466-06a67b3143f0"", ""411f...","[""[whole interspersed with hand drawn pen deco...",[1.0],"[""Catherine1""]","[""2016-07-15 20:58:25"", ""2016-07-15 21:01:08""]"
3,300072,https://catalog.archives.gov/id/300072,Illustrated Family Record (Fraktur) Found in R...,<NA>,"[{""number"": ""NWCTB-15-NM22E19-W15214-SLIDE59"",...",48118979,https://s3.amazonaws.com/NARAprodstorage/opast...,"[""48118980""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),"[""20747713""]","[""'-\n-\n-\n---\n---\nto\n-\n-\nto\n\u2117\nAu...","[""2024-11-22T02:30:45.000Z""]",FamilySearch,"[""88200bb2-dc85-3012-9476-7384aed13474""]","[""FAMILY RECORD\nO'Cranby's C.P.S.\n...",[9.0],"[""mpage77""]","[""2020-10-25 08:53:51""]"
4,300092,https://catalog.archives.gov/id/300092,Fraktur of the Family of Peter Hunt,<NA>,"[{""note"": ""This is the NARA Internal Exhibit T...",<NA>,<NA>,"[""16411580""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),"[""20747714""]","[""I\n8\nto\nSarah Cary\nPeter Hunt-\nHannah Hu...","[""2024-11-22T02:30:45.000Z""]",FamilySearch,"[""f2fdfba6-1c0f-37dd-9f12-37cc6e28dcff""]","[""Births. marriages. ...",[2.0],"[""ScribeOfPoland""]","[""2016-08-10 05:06:39""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78921,245050683,https://catalog.archives.gov/id/245050683,Copy of the Affidavit of Samuel Cherry of Pari...,1807-02-26,"[{""number"": ""HD1-730357612"", ""type"": ""HMS Asse...",267322352,https://s3.amazonaws.com/NARAprodstorage/lz/dc...,"[""267322353"", ""267322354"", ""267322355""]","[""https://s3.amazonaws.com/NARAprodstorage/lz/...",Image (JPG),"[""20747693"", ""20747694"", ""20747695""]","[""I Samuel Cherry now of the County of Oneida ...","[""2024-11-22T02:30:45.000Z""]",FamilySearch,"[""8b78a52d-7625-4bde-81c4-8dbb7a2a3fee"", ""be95...","[""I Samuel Cherry now of the County of Oneida ...","[3.0, 3.0, 2.0]","[""mlundblad;engadean390608;NPSkjwojack"", ""mlun...","[""2025-01-23 19:12:54"", ""2024-03-26 03:07:15"",..."
78922,245050686

In [27]:
output_path = 'nara_pension_file_pages_by_naid.parquet'
df_to_save.to_parquet(output_path, engine='fastparquet')  # or engine='pyarrow'
print(f"Saved collapsed dataframe to {output_path}")

Saved collapsed dataframe to nara_pension_file_pages_by_naid.parquet


In [29]:
# Build and save list of NAIDs where BOTH ocrText and transcriptionText are blank/missing
import json

def _is_blank_json_array(s: object) -> bool:
    if pd.isna(s):
        return True
    s = str(s).strip()
    return s == "" or s == "[]"

mask = df_to_save['ocrText'].apply(_is_blank_json_array) & df_to_save['transcriptionText'].apply(_is_blank_json_array)

naids_both_blank = (
    df_to_save.loc[mask, 'NAID']
      .dropna()
      .astype('int64')
      .unique()
      .tolist()
)

print(f"Rows with both blank: {mask.sum()} of {len(df_to_save)} ({mask.mean()*100:.2f}%)")
print(f"Unique NAIDs with both blank: {len(naids_both_blank)}")

with open('naids_both_blank.json', 'w') as f:
    json.dump(naids_both_blank, f)
print("Saved NAIDs to naids_both_blank.json")


Rows with both blank: 111 of 78926 (0.14%)
Unique NAIDs with both blank: 111
Saved NAIDs to naids_both_blank.json


In [30]:
# Verify NAIDs in naids_both_blank.json are truly blank in original df
import json

# Load NAIDs
with open('naids_both_blank.json', 'r') as f:
    naids_list = json.load(f)

# Helper for original df (treat NaN or empty string as blank)
def _is_blank_text(v):
    if v is None:
        return True
    try:
        if pd.isna(v):
            return True
    except Exception:
        pass
    return isinstance(v, str) and v.strip() == ''

# Ensure NAID is numeric for matching (does not mutate df permanently if already set)
df['NAID'] = pd.to_numeric(df['NAID'], errors='coerce').astype('Int64')

subset = df[df['NAID'].isin(naids_list)].copy()

# Per-row mask: both blank
row_both_blank = subset['ocrText'].map(_is_blank_text) & subset['transcriptionText'].map(_is_blank_text)

# Per-NAID validation: all rows both blank
per_naid = (
    subset.assign(_both=row_both_blank)
          .groupby('NAID', dropna=False)
          .agg(
              rows=('NAID', 'size'),
              all_both_blank=('_both', 'all'),
              any_nonblank=('_both', lambda s: (~s).any()),
          )
          .reset_index()
)

num_naids = len(naids_list)
num_checked = per_naid.shape[0]
num_all_ok = int(per_naid['all_both_blank'].sum())
num_fail = int((~per_naid['all_both_blank']).sum())

print(f"NAIDs in file: {num_naids}")
print(f"Found in original df: {num_checked}")
print(f"All rows blank for: {num_all_ok} NAIDs")
print(f"Failures (have at least one non-blank row): {num_fail}")

if num_fail > 0:
    bad_ids = per_naid.loc[~per_naid['all_both_blank'], 'NAID'].tolist()
    print("\nSample failing NAIDs:", bad_ids[:10])
    # Show a few offending rows for the first failing NAID
    bad = subset[subset['NAID'] == bad_ids[0]].copy()
    bad['_both_blank'] = bad['ocrText'].map(_is_blank_text) & bad['transcriptionText'].map(_is_blank_text)
    display(bad[['NAID', 'pageObjectId', 'ocrText', 'transcriptionText', '_both_blank']].head(20))


NAIDs in file: 111
Found in original df: 111
All rows blank for: 111 NAIDs
Failures (have at least one non-blank row): 0


'["(3 - 1776)\\nSERVICE\\nConn\\nCONTENTS\\nnessee\\nAbbott, Asa\\nElizabeth\\nNUMBER\\nW. 20558\\n19", "Declaration\\nW\\nOne\\nthe order to obtain the benefits of the provisions made by the\\nact of Congress passed July 7th 1838, March 8th 1843. and\\nState of New York\\n17th June 1844: Grantly pensions to certain widows\\nOnondaga County \\u23ac ss On this 23 day of August - 1845\\nbefore the subscriber one of the Judges in and for the County\\naforesaid personally appeared Elizabeth Abbott of the town of\\nClay County of Onondaga and State of New York aged (81) Eighty four\\nyears the 26th day of July 1845. who being first duly sworn\\naccording to law doth on her oath make the following Decla\\nration in order to obtain the benefits of the provisions made\\nby the acts of Congress\\nand July 19th 1844,\\npassed July 7th 1838, March 3d 1833\\ncertain widows,\\nentitled act granting pensions to\\nThat she is the widow of Asa Abbott who\\nwas formerly a resident of Lebanon in the Cou

In [44]:
# Are all original titles present in the NAID-collapsed parquet?
import json

orig_path = 'nara_pension_file_pages.parquet'
collapsed_path = 'nara_pension_file_pages_by_naid.parquet'

orig_df = pd.read_parquet(orig_path, engine='fastparquet')
coll_df = pd.read_parquet(collapsed_path, engine='fastparquet')

# Normalize collapsed titles if JSON-serialized arrays
def _norm_title(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if (s.startswith('[') and s.endswith(']')) or (s.startswith('{') and s.endswith('}')):
        try:
            obj = json.loads(s)
            if isinstance(obj, list):
                return ' | '.join(str(x) for x in obj)
            return str(obj)
        except Exception:
            return s
    return s

orig_titles = orig_df['title'].dropna().astype(str).str.strip()
coll_titles = coll_df['title'].dropna().apply(_norm_title).dropna().astype(str).str.strip()

orig_set = set(orig_titles.unique())
coll_set = set(coll_titles.unique())

missing_from_collapsed = sorted(orig_set - coll_set)  # titles in original not found in collapsed

print(f"Unique titles in original: {len(orig_set)}")
print(f"Unique titles in collapsed: {len(coll_set)}")
print(f"Titles present in original but NOT in collapsed: {len(missing_from_collapsed)}")

# Show a small sample if any missing
if missing_from_collapsed:
    print("\nSample missing titles (up to 20):")
    for t in missing_from_collapsed[:20]:
        print(f"- {t}")
else:
    print("All original titles are present in the collapsed parquet.")


Unique titles in original: 76804
Unique titles in collapsed: 76804
Titles present in original but NOT in collapsed: 0
All original titles are present in the collapsed parquet.


In [48]:
# display(df_to_save.loc[100]['ocrText'])
display(df_to_save.loc[100]['title'])

'Revolutionary War Pension and Bounty Land Warrant Application File W. 20558, Asa Abbott, Conn'

In [49]:
import json
import re

def _norm_title(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if (s.startswith('[') and s.endswith(']')) or (s.startswith('{') and s.endswith('}')):
        try:
            obj = json.loads(s)
            if isinstance(obj, list):
                return ' | '.join(str(x) for x in obj)
            return str(obj)
        except Exception:
            return s
    return s

assert 'df_to_save' in globals(), "df_to_save not found. Run the cell that builds df_to_save first."

cdf = df_to_save.copy()
cdf['title_norm'] = cdf['title'].apply(_norm_title)

mask = cdf['title_norm'].fillna('').str.contains(r'\bFile\s+R\.', case=False, regex=True)
display(cdf.loc[mask, ['NAID', 'title_norm']].head(5))

,NAID,title_norm
59,53774495,Revolutionary War Pension and Bounty Land Warr...
80,53775312,Revolutionary War Pension and Bounty Land Warr...
95,53775622,Revolutionary War Pension and Bounty Land Warr...
141,53777351,Revolutionary War Pension and Bounty Land Warr...
144,53777491,Revolutionary War Pension and Bounty Land Warr...


In [51]:
display(df_to_save.loc[59]['ocrText'])
display(df_to_save.loc[59]['title'])

'["(3 - 1776)\\nSERVICE\\nConn\\nCONTENTS\\nAbbey, Joseph James\\nJames\\nor\\n\\u25cf \\u25cf \\u25cf\\nNUMBER\\nR. 1.", "State of New York\\nDelaware County\\nsix\\n2\\n2\\nOn this ninth day of January 1837 personly\\nappeared before the Honl Amos Daughter\\nEsquire one of the Judges of the County Courts\\nwithin and for the County of Delaware\\nJames Abbey\\nresident of the Town of Sidney in the\\n---\\nCounty of Delaware in the State of New York aged seventy\\nyears and six months who being first duly sworn according to\\nlaw doth on his oath make the following declaration in\\norder to obtain the benefit of the provision made by the act\\nof Congress passed the 7th June 1832 - That he enlisted did\\nprivate soldier in the Army of the United States he thinks in the\\nyear 1780 or 1781 for the Term of three years in the Connecticut line\\nunder the following named officers viz that he enlisted at the\\nTown of Preston in Connecticut in the month of May either\\nin the year 1778 or 1

'Revolutionary War Pension and Bounty Land Warrant Application File R. 1, Joseph James Abbey, Conn.'

In [52]:
import json
import re

def _norm_title(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if (s.startswith('[') and s.endswith(']')) or (s.startswith('{') and s.endswith('}')):
        try:
            obj = json.loads(s)
            if isinstance(obj, list):
                return ' | '.join(str(x) for x in obj)
            return str(obj)
        except Exception:
            return s
    return s

assert 'df_to_save' in globals(), "df_to_save not found. Run the cell that builds df_to_save first."

cdf = df_to_save.copy()
cdf['title_norm'] = cdf['title'].apply(_norm_title)

mask = cdf['title_norm'].fillna('').str.contains(r'\bFile\s+S\.', case=False, regex=True)
display(cdf.loc[mask, ['NAID', 'title_norm']].head(5))

,NAID,title_norm
46,53774215,Revolutionary War Pension and Bounty Land Warr...
47,53774236,Revolutionary War Pension and Bounty Land Warr...
51,53774302,Revolutionary War Pension and Bounty Land Warr...
56,53774451,Revolutionary War Pension and Bounty Land Warr...
57,53774479,Revolutionary War Pension and Bounty Land Warr...


In [54]:
display(df_to_save.loc[46]['ocrText'])
display(df_to_save.loc[46]['title'])
display(df_to_save.loc[46]['NAID'])


'["Va\\nSERVICE\\nCONTENTS\\nAarons, Abraham\\nNUMBER\\nS. 30,813", "743\\nKentucky\\nAbraham Barons\\nof Adair Co in the State of Kentucky\\nwho was a private in the company\\ncommanded\\nby Captain Mitcher of the Re\\ncommanded\\nby Col Mason in the\\nline for\\nyear six with\\nInscribed on the Roll of Kentucky\\nat the rate of 40 Dollars - Cents per annum,\\nto commence on the 4th day of March, 1831.\\nCertificate of Pension issued the 12 day of April\\n1833 and sent to Wm\\nOwens Columbia H\\nArrears to the 4th of March\\n80 00\\nSemi-anl. allowance ending 4 Sept\\n200\\n$100.00\\nRevolutionary Claim,\\nAct June 7, 1832.\\nRecorded by Daniel Ray & Clerk,\\nBook\\nVol.", "see letter to\\nWm Owens, Columbia\\n18 April 1835\\nPen Agent", "of\\nAbraham Aaron\\n21624\\n2\\nof\\n-\\n\\u25cf\\n2\\nthe\\nA.\\nof\\nAarons\\nDeclaration", "State of Kentucky Adair County Sct\\non this\\n29th day of November 1832 Abraham\\nAarons a resident of the County of Adair of\\nand State of Kentucky age

'Revolutionary War Pension and Bounty Land Warrant Application File S. 30. 813, Abraham Aarons, Va.'

np.int64(53774215)

In [55]:
# Find 5 examples with "File B.", "BLW", or "B L Wt" in title
import json
import re

def _norm_title(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if (s.startswith('[') and s.endswith(']')) or (s.startswith('{') and s.endswith('}')):
        try:
            obj = json.loads(s)
            if isinstance(obj, list):
                return ' | '.join(str(x) for x in obj)
            return str(obj)
        except Exception:
            return s
    return s

assert 'df_to_save' in globals(), "df_to_save not found. Run the cell that builds df_to_save first."

cdf = df_to_save.copy()
cdf['title_norm'] = cdf['title'].apply(_norm_title)

# Search for patterns: "File B.", "BLW", or "B L Wt"
pattern1 = r'\bFile\s+B\.'
pattern2 = r'\bBLW\b'
pattern3 = r'\bB\s+L\s+Wt\b'

mask = (cdf['title_norm'].fillna('').str.contains(pattern1, case=False, regex=True) |
        cdf['title_norm'].fillna('').str.contains(pattern2, case=False, regex=True) |
        cdf['title_norm'].fillna('').str.contains(pattern3, case=False, regex=True))

print(f"Found {mask.sum()} rows matching 'File B.', 'BLW', or 'B L Wt' patterns")
display(cdf.loc[mask, ['NAID', 'title_norm']].head(5))


Found 3198 rows matching 'File B.', 'BLW', or 'B L Wt' patterns


,NAID,title_norm
49,53774273,Revolutionary War Pension and Bounty Land Warr...
106,53776045,Revolutionary War Pension and Bounty Land Warr...
127,53776877,Revolutionary War Pension and Bounty Land Warr...
135,53777158,Revolutionary War Pension and Bounty Land Warr...
162,53777957,Revolutionary War Pension and Bounty Land Warr...


In [57]:
# Find 5 examples with "OW" or "old war" in title
import json
import re

def _norm_title(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if (s.startswith('[') and s.endswith(']')) or (s.startswith('{') and s.endswith('}')):
        try:
            obj = json.loads(s)
            if isinstance(obj, list):
                return ' | '.join(str(x) for x in obj)
            return str(obj)
        except Exception:
            return s
    return s

assert 'df_to_save' in globals(), "df_to_save not found. Run the cell that builds df_to_save first."

cdf = df_to_save.copy()
cdf['title_norm'] = cdf['title'].apply(_norm_title)

# Search for patterns: "OW" or "old war"
pattern1 = r'\bOW\b'
pattern2 = r'\bold\s+war\b'

mask = (cdf['title_norm'].fillna('').str.contains(pattern1, case=False, regex=True) |
        cdf['title_norm'].fillna('').str.contains(pattern2, case=False, regex=True))

print(f"Found {mask.sum()} rows matching 'OW' or 'old war' patterns")
display(cdf.loc[mask, ['NAID', 'title_norm']].head(5))


Found 47 rows matching 'OW' or 'old war' patterns


,NAID,title_norm
1852,53830671,Revolutionary War Pension and Bounty Land Warr...
2082,53837916,Revolutionary War Pension and Bounty Land Warr...
7845,54010904,Revolutionary War Pension and Bounty Land Warr...
8378,54025906,Revolutionary War Pension and Bounty Land Warr...
10580,54093258,Revolutionary War Pension and Bounty Land Warr...


In [59]:
# Find 5 examples with "N A Acc" (National Archives Accession) in title
import json
import re

def _norm_title(v):
    if pd.isna(v):
        return pd.NA
    s = str(v).strip()
    if (s.startswith('[') and s.endswith(']')) or (s.startswith('{') and s.endswith('}')):
        try:
            obj = json.loads(s)
            if isinstance(obj, list):
                return ' | '.join(str(x) for x in obj)
            return str(obj)
        except Exception:
            return s
    return s

assert 'df_to_save' in globals(), "df_to_save not found. Run the cell that builds df_to_save first."

cdf = df_to_save.copy()
cdf['title_norm'] = cdf['title'].apply(_norm_title)

# Search for patterns: "N A Acc" or variations
pattern1 = r'\bN\s*A\s*Acc\b'
pattern2 = r'\bNational\s+Archives\s+Accession\b'
pattern3 = r'\bNo\s+pension\s+file\s+found\b'

mask = (cdf['title_norm'].fillna('').str.contains(pattern1, case=False, regex=True) |
        cdf['title_norm'].fillna('').str.contains(pattern2, case=False, regex=True) |
        cdf['title_norm'].fillna('').str.contains(pattern3, case=False, regex=True))

print(f"Found {mask.sum()} rows matching 'N A Acc', 'National Archives Accession', or 'No pension file found' patterns")
display(cdf.loc[mask, ['NAID', 'title_norm']].head(5))


Found 0 rows matching 'N A Acc', 'National Archives Accession', or 'No pension file found' patterns


,NAID,title_norm


In [58]:
display(df_to_save.loc[1852]['ocrText'])
display(df_to_save.loc[1852]['title'])
display(df_to_save.loc[1852]['NAID'])

'["1776)\\nSERVICE\\nNot Rev. War\\nN.W. Ind. War 1791\\nCONTENTS\\nApplegate, James\\nKy. Agency\\nwere\\nNUMBER\\nOld War Inv. File 6522", "Ca\\nby\\n1207.\\nKentucky\\nJames Applegate,\\nSergeant\\ntain Jo: Brock\'s company of the 1st Reg\\nS. Levies -\\nRatio of disability, Three Fourth\\nDate of discharge,\\nTime of closing testimony, 8th Sept 1826\\n\\u2503 ascribed\\nat\\n8th\\n-\\non the Roll of Kentucky\\nC\\ndollars per month, to commence on the\\nSeptember 1826\\nCertificate of pension issued the 8th of Nov. 1826.\\nand sent to George W. Botts\\nFlemingsburg, Ky.\\n-\\nArrears to 4th Mar 1827 $34.94\\nSemi-anl all\'ce ending\\nto\\nBy\\n34.94 \\u2503\\nAct of the 4th Feb. 1822.", "A\\nhe\\nAg\\nC\\nto\\nI\\nI\\nJames Applegate estate on oath, the reasons\\nnot apply for a Pension before, are while younger in life\\nwas able to make a living for himself by labour since\\nhas come on he has been lessable, and while he\\nwould get his living in any way without being chargable\\

'Revolutionary War Pension and Bounty Land Warrant Application File Old War Ind. File 6522, James Applegate, Not Rev. War N. W. Ind. War 1791'

np.int64(53830671)

In [1]:
# Load and display the processed data with date classifications
import pandas as pd

# Load the processed data
df_processed = pd.read_parquet('nara_pension_file_pages_titles_dates_parsed.parquet')

print(f"📊 Processed data shape: {df_processed.shape}")
print(f"📋 Columns: {list(df_processed.columns)}")

print("\n" + "="*80)
print("🔍 FIRST 10 ROWS")
print("="*80)
display(df_processed.head(10))

print("\n" + "="*80)
print("🔍 LAST 10 ROWS")
print("="*80)
display(df_processed.tail(10))

# Show summary statistics
print("\n" + "="*80)
print("📈 SUMMARY STATISTICS")
print("="*80)
print(f"File type distribution:")
print(df_processed['file_type_category'].value_counts())

print(f"\nDate classification summary:")
print(f"Rows with application dates: {(df_processed['application_dates'].apply(len) > 0).sum()}")
print(f"Rows with service dates: {(df_processed['service_dates'].apply(len) > 0).sum()}")
print(f"Total application dates: {sum(len(dates) for dates in df_processed['application_dates'])}")
print(f"Total service dates: {sum(len(dates) for dates in df_processed['service_dates'])}")
print(f"Total other dates: {sum(len(dates) for dates in df_processed['other_dates'])}")


📊 Processed data shape: (100, 30)
📋 Columns: ['NAID', 'naraURL', 'title', 'logicalDate', 'variantControlNumbers', 'pdfObjectID', 'pdfURL', 'pageObjectId', 'pageURL', 'pageImageType', 'ocrID', 'ocrText', 'ocrUploadDate', 'ocrContributor', 'transcriptionID', 'transcriptionText', 'transcriptionContributionCount', 'transcriptionUserNames', 'transcriptionDate', 'parsed_title_json', 'applicant_from_title', 'applicant_place_from_title', 'file_type_category', 'file_type_category_info', 'extracted_dates_json', 'extracted_dates', 'application_dates', 'service_dates', 'other_dates', 'date_confidence_scores']

🔍 FIRST 10 ROWS


,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,...,applicant_from_title,applicant_place_from_title,file_type_category,file_type_category_info,extracted_dates_json,extracted_dates,application_dates,service_dates,other_dates,date_confidence_scores
0,300036,https://catalog.archives.gov/id/300036,Illustrated Family Record (Fraktur) Found in R...,None,"[{""number"": ""NWCTB-15-NM22E19-R11789-SLIDE115""...",28238881,https://s3.amazonaws.com/NARAprodstorage/lz/re...,"[""14989172""]","[""https://s3.amazonaws.com/NARAprodstorage/lz/...",Image (JPG),...,Illustrated Family Record (Fraktur) Found in R...,for Joseph Wood,,"{""matched_snippet"": """", ""normalized_category"":...","[""1762"", ""1781"", ""1781"", ""1789"", ""1794"", ""1797...","[1762, 1781, 1781, 1789, 1794, 1797, 1799, 180...",[],"[1781, 1781]","[1762, 1789, 1794, 1797, 1799, 1800, 1806, 181...","{""1762"": 0.0, ""1781"": 0.4, ""1789"": 0.0, ""1794""..."
1,300040,https://catalog.archives.gov/id/300040,Illustrated Family Record (Fraktur) Found in R...,None,"[{""number"": ""NWCTB-15-NM22E19-R5568-SLIDE27"", ...",24519842,https://s3.amazonaws.com/NARAprodstorage/lz/re...,"[""24519840"", ""24519841""]","[""https://s3.amazonaws.com/NARAprodstorage/lz/...",Image (JPG),...,Illustrated Family Record (Fraktur) Found in R...,for John Jenkins,,"{""matched_snippet"": """", ""normalized_category"":...","[""1755"", ""1765"", ""1794"", ""1795"", ""1796"", ""1797...","[1755, 1765, 1794, 1795, 1796, 1797, 1800, 180...",[],[],"[1755, 1765, 1794, 1795, 1796, 1797, 1800, 180...","{""1755"": 0.0, ""1765"": 0.0, ""1794"": 0.0, ""1795""..."
2,300070,https://catalog.archives.gov/id/300070,Illustrated Family Record (Fraktur) Found in R...,None,"[{""note"": ""This is the NARA internal exhibit t...",14990014,https://s3.amazonaws.com/NARAprodstorage/opast...,"[""14990013"", ""14990015""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Illustrated Family Record (Fraktur) Found in R...,for David Rogers,,"{""matched_snippet"": """", ""normalized_category"":...","[""1760"", ""1769"", ""1787"", ""1788"", ""1789"", ""1789...","[1760, 1769, 1787, 1788, 1789, 1789, 1792, 179...",[],[],"[1760, 1769, 1787, 1788, 1789, 1789, 1792, 179...","{""1760"": 0.0, ""1769"": 0.0, ""1787"": 0.0, ""1788""..."
3,300072,https://catalog.archives.gov/id/300072,Illustrated Family Record (Fraktur) Found in R...,None,"[{""number"": ""NWCTB-15-NM22E19-W15214-SLIDE59"",...",48118979,https://s3.amazonaws.com/NARAprodstorage/opast...,"[""48118980""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Illustrated Family Record (Fraktur) Found in R...,for Reuben Phelps,,"{""matched_snippet"": """", ""normalized_category"":...","[""1747"", ""1759"", ""1776"", ""1777"", ""1778"", ""1780...","[1747, 1759, 1776, 1777, 1778, 1780, 1782, 178...",[],"[1776, 1777, 1778, 1780, 1782, 1783]","[1747, 1759, 1799, 1806, 1810, 1824]","{""1747"": 0.0, ""1759"": 0.0, ""1776"": 0.4, ""1777""..."
4,300092,https://catalog.archives.gov/id/300092,Fraktur of the Family of Peter Hunt,None,"[{""note"": ""This is the NARA Internal Exhibit T...",None,None,"[""16411580""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Fraktur of the Family of Peter Hunt,,,"{""matched_snippet"": """", ""normalized_category"":...","[""1726"", ""1744"", ""1757"", ""1761"", ""1779"", ""1779...","[1726, 1744, 1757, 1761, 1779, 1779, 1781, 178...",[],"[1779, 1779, 1781]","[1726, 1744, 1757, 1761, 1784, 1788, 1792, 179...","{""1726"": 0.0, ""1744"": 0.0, ""1757"": 0.0, ""1761""..."
5,300134,https://catalog.archives.gov/id/300134,Illustrated family record (Fraktur) found in R...,None,"[{""number"": ""NWCTB-15-NM22E19-W19867-SLIDE34"",...",None,None,"[""14988263""]","[""https://s3.amazonaws.com/NARAprodstorage/lz/...",Image (TIFF),...,Illustrated family record (Fraktur) found in R...,for John McChesney,,"{""matched_snippet"": """", ""normalized_category"":...","[""1764


🔍 LAST 10 ROWS


,NAID,naraURL,title,logicalDate,variantControlNumbers,pdfObjectID,pdfURL,pageObjectId,pageURL,pageImageType,...,applicant_from_title,applicant_place_from_title,file_type_category,file_type_category_info,extracted_dates_json,extracted_dates,application_dates,service_dates,other_dates,date_confidence_scores
90,53775486,https://catalog.archives.gov/id/53775486,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775487"", ""53775488"", ""53775489"", ""53775490...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Revolutionary War Pension and Bounty Land Warr...,973,,"{""matched_snippet"": """", ""normalized_category"":...","[""1776""]",[1776],[],[1776],[],"{""1776"": 0.4}"
91,53775517,https://catalog.archives.gov/id/53775517,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775518""]","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Moses Abbot,Continental N. H,T,"{""matched_snippet"": ""File T. 16592,"", ""normali...",[],[],[],[],[],{}
92,53775519,https://catalog.archives.gov/id/53775519,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775520"", ""53775521"", ""53775522"", ""53775523...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,502,Nathaniel Chandler Abbot,W,"{""matched_snippet"": ""File W. 15,"", ""normalized...",[],[],[],[],[],{}
93,53775559,https://catalog.archives.gov/id/53775559,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775560"", ""53775561"", ""53775562"", ""53775563...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Reuben Abbot,N. H,S,"{""matched_snippet"": ""File S. 17808,"", ""normali...",[],[],[],[],[],{}
94,53775571,https://catalog.archives.gov/id/53775571,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775572"", ""53775573"", ""53775574"", ""53775575...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Timothy Abbot,N H,W,"{""matched_snippet"": ""File W. 20560,"", ""normali...","[""1776""]",[1776],[],[1776],[],"{""1776"": 0.4}"
95,53775622,https://catalog.archives.gov/id/53775622,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775623"", ""53775624"", ""53775625"", ""53775626...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,334,William Abbot,R,"{""matched_snippet"": ""File R. 11,"", ""normalized...","[""1776""]",[1776],[],[1776],[],"{""1776"": 0.4}"
96,53775671,https://catalog.archives.gov/id/53775671,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775672"", ""53775673"", ""53775674"", ""53775675...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,932,Samuel Abbot,S,"{""matched_snippet"": ""File S. 11,"", ""normalized...","[""1833""]",[1833],[1833],[],[],"{""1833"": 0.6000000000000001}"
97,53775690,https://catalog.archives.gov/id/53775690,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775691"", ""53775692"", ""53775693"", ""53775694...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JPG),...,Revolutionary War Pension and Bounty Land Warr...,,,"{""matched_snippet"": """", ""normalized_category"":...",[],[],[],[],[],{}
98,53775751,https://catalog.archives.gov/id/53775751,Revolutionary War Pension and Bounty Land Warr...,None,"[{""number"": ""Fold3 2016"", ""type"": ""Search Iden...",None,None,"[""53775752"", ""53775753"", ""53775754"", ""53775755...","[""https://s3.amazonaws.com/NARAprodstorage/opa...",Image (JP


📈 SUMMARY STATISTICS
File type distribution:
file_type_category
       59
S      21
W      15
R       3
BLW     1
T       1
Name: count, dtype: int64

Date classification summary:
Rows with application dates: 5
Rows with service dates: 54
Total application dates: 10
Total service dates: 103
Total other dates: 344


In [ ]:
from non_llm_parsing import process_deterministic_only

print("Loading full dataset...")
df = pd.read_parquet('nara_pension_file_pages_by_naid.parquet')
print(f"Loaded {len(df)} rows")

# Process entire dataset
print(f"Processing entire dataset with {len(df)} rows...")

# Process deterministically
processed_df = process_deterministic_only(df)

print("\nResults:")
print(f"File type categories: {processed_df['file_type_category'].value_counts().to_dict()}")
print(f"Total dates extracted: {sum(len(dates) for dates in processed_df['extracted_dates'])}")
print(f"Rows with dates: {(processed_df['extracted_dates'].apply(len) > 0).sum()}")

# Count date types
app_dates_count = sum(len(dates) for dates in processed_df['application_dates'])
service_dates_count = sum(len(dates) for dates in processed_df['service_dates'])
other_dates_count = sum(len(dates) for dates in processed_df['other_dates'])
print(f"Application dates: {app_dates_count}")
print(f"Service dates: {service_dates_count}")
print(f"Other dates: {other_dates_count}")

# Show some examples
print("\nSample results:")
for i in range(min(3, len(processed_df))):
    row = processed_df.iloc[i]
    print(f"Row {i}:")
    print(f"  Title: {row['title'][:60]}...")
    print(f"  File type: {row['file_type_category']}")
    print(f"  Total dates: {len(row['extracted_dates'])}")
    if len(row['extracted_dates']) > 0:
        print(f"    Application: {row['application_dates']}")
        print(f"    Service: {row['service_dates']}")
        print(f"    Other: {row['other_dates']}")
    print()

# Save the processed dataframe
output_file = 'nara_pension_file_pages_titles_dates_parsed.parquet'
processed_df.to_parquet(output_file)
print(f"\n💾 Saved processed data to: {output_file}")
print(f"📊 Final shape: {processed_df.shape}")